# EE 451: Communications Systems
## Lecture 27 - Digital Performance Analysis: BER & Matched Filtering

### Learning Objectives

By the end of this notebook, you will be able to:

1. Derive Bit Error Rate (BER) for BPSK in AWGN channel
2. Apply matched filter theory for optimal detection
3. Represent signals in signal space using orthonormal basis functions
4. Design correlation receivers for digital modulation
5. Calculate and compare BER for ASK, FSK, and BPSK
6. Simulate BER vs. SNR curves in Python

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc, erf
from scipy import signal

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Define Q-function
def Q_function(x):
    """Q-function: Tail probability of standard normal distribution"""
    return 0.5 * erfc(x / np.sqrt(2))

print("Libraries imported successfully!")
print("Q-function defined: Q(x) = 0.5 * erfc(x/√2)")

## Part 1: Matched Filter Theory

The **matched filter** is the optimal linear filter for maximizing SNR in the presence of AWGN.

### Matched Filter Definition
For a signal $s(t)$, the matched filter impulse response is:
$$h(t) = s(T - t)$$

where $T$ is the symbol duration (time-reversed signal).

### Output SNR
The matched filter maximizes the output SNR:
$$\text{SNR}_{\text{out}} = \frac{2E_s}{N_0}$$

where:
- $E_s$ = Signal energy
- $N_0$ = Noise power spectral density

In [ ]:
# Demonstrate matched filter
fs = 10000  # Sampling frequency (Hz)
T = 0.01  # Pulse duration (10 ms)
t = np.arange(0, T, 1/fs)

# Signal: Rectangular pulse
s = np.ones(len(t))

# Matched filter: Time-reversed signal
h = s[::-1]

# Add noise
snr_dB = 10
snr_linear = 10**(snr_dB / 10)
E_s = np.sum(s**2) / fs  # Signal energy
N0 = 2 * E_s / snr_linear
noise_std = np.sqrt(N0 * fs / 2)
noise = noise_std * np.random.randn(len(t))

# Received signal
r = s + noise

# Matched filter output
y = np.correlate(r, h, mode='same') / fs

# Unmatched filter (simple integration)
y_unmatched = np.cumsum(r) / fs

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Signal and matched filter
axes[0, 0].plot(t * 1000, s, linewidth=2, label='Signal s(t)')
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Transmitted Signal')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(t * 1000, h, linewidth=2, color='red', label='Matched Filter h(t) = s(T-t)')
axes[0, 1].set_xlabel('Time (ms)')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].set_title('Matched Filter (Time-Reversed Signal)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Received signal (noisy)
axes[1, 0].plot(t * 1000, r, linewidth=1, alpha=0.7, label=f'Received (SNR = {snr_dB} dB)')
axes[1, 0].plot(t * 1000, s, '--', linewidth=2, color='green', alpha=0.5, label='Original')
axes[1, 0].set_xlabel('Time (ms)')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title('Received Signal (with AWGN)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Matched filter output vs unmatched
axes[1, 1].plot(t * 1000, y, linewidth=2, label='Matched Filter Output')
axes[1, 1].plot(t * 1000, y_unmatched, linewidth=2, alpha=0.7, label='Integrator (Unmatched)')
decision_time = len(t) // 2
axes[1, 1].axvline(x=t[decision_time] * 1000, color='red', linestyle='--', 
                   linewidth=2, label='Decision Time')
axes[1, 1].set_xlabel('Time (ms)')
axes[1, 1].set_ylabel('Amplitude')
axes[1, 1].set_title('Filter Outputs (Peak at Decision Time)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate SNR improvement
snr_out_matched = np.max(y)**2 / np.var(y - np.max(y))
snr_out_unmatched = np.max(y_unmatched)**2 / np.var(y_unmatched - np.max(y_unmatched))

print("\nMatched Filter Performance:")
print("═" * 70)
print(f"Input SNR: {snr_dB} dB")
print(f"Theoretical output SNR: {10*np.log10(2*E_s/N0):.1f} dB")
print(f"Matched filter output SNR: {10*np.log10(snr_out_matched):.1f} dB")
print(f"Unmatched filter output SNR: {10*np.log10(snr_out_unmatched):.1f} dB")
print(f"\nSNR improvement: {10*np.log10(snr_out_matched/snr_out_unmatched):.1f} dB")
print("═" * 70)
print("\nKey Insight: Matched filter maximizes SNR at decision instant")

## Part 2: Signal Space Representation

Signals can be represented as vectors in **signal space** using orthonormal basis functions.

### BPSK Signal Space
For BPSK with carrier $\cos(2\pi f_c t)$:

**Basis function:**
$$\phi(t) = \sqrt{\frac{2}{T}} \cos(2\pi f_c t)$$

**Signals:**
- Bit "1": $s_1(t) = A\cos(2\pi f_c t) \rightarrow s_1 = +A\sqrt{T/2}$
- Bit "0": $s_0(t) = -A\cos(2\pi f_c t) \rightarrow s_0 = -A\sqrt{T/2}$

**Euclidean distance:**
$$d = |s_1 - s_0| = 2A\sqrt{T/2} = A\sqrt{2T}$$

In [ ]:
# BPSK signal space visualization
A = 1  # Amplitude
T_b = 0.001  # Bit duration (1 ms)
f_c = 5000  # Carrier frequency (5 kHz)

# Energy per bit
E_b = (A**2 * T_b) / 2

# Signal space coordinates
s1 = A * np.sqrt(T_b / 2)  # Bit 1
s0 = -A * np.sqrt(T_b / 2)  # Bit 0

# Euclidean distance
distance = np.abs(s1 - s0)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Signal space diagram
axes[0].plot([0, s1], [0, 0], 'o-', linewidth=3, markersize=15, color='blue', label='s₁ (Bit 1)')
axes[0].plot([0, s0], [0, 0], 'o-', linewidth=3, markersize=15, color='red', label='s₀ (Bit 0)')
axes[0].axvline(x=0, color='green', linestyle='--', linewidth=2, label='Decision Boundary')
axes[0].annotate('', xy=(s1, 0.001), xytext=(s0, 0.001),
                arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
axes[0].text(0, 0.0015, f'd = {distance:.4f}', ha='center', fontsize=12,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

axes[0].set_xlabel('φ₁(t) = √(2/T) cos(2πf_c t)', fontsize=12)
axes[0].set_ylabel('Amplitude', fontsize=12)
axes[0].set_title('BPSK Signal Space (1-Dimensional)', fontsize=14)
axes[0].grid(True, alpha=0.3)
axes[0].legend()
axes[0].set_xlim(-0.04, 0.04)
axes[0].set_ylim(-0.002, 0.002)
axes[0].axhline(y=0, color='k', linewidth=0.5)

# Decision regions with noise
# Simulate received signals with noise
num_samples = 1000
snr_dB_demo = 10
snr_linear_demo = 10**(snr_dB_demo / 10)
sigma = np.sqrt(E_b / (2 * snr_linear_demo))

# Received signals (add noise)
r1 = s1 + sigma * np.random.randn(num_samples)  # When bit 1 sent
r0 = s0 + sigma * np.random.randn(num_samples)  # When bit 0 sent

axes[1].scatter(r1, np.random.randn(num_samples) * 0.0001, alpha=0.3, s=10, 
               color='blue', label='Received (Bit 1 sent)')
axes[1].scatter(r0, np.random.randn(num_samples) * 0.0001, alpha=0.3, s=10, 
               color='red', label='Received (Bit 0 sent)')
axes[1].axvline(x=0, color='green', linestyle='--', linewidth=2, label='Decision Boundary')

# Mark true symbols
axes[1].plot(s1, 0, 'b*', markersize=20, label='True s₁')
axes[1].plot(s0, 0, 'r*', markersize=20, label='True s₀')

axes[1].set_xlabel('Received Signal Projection', fontsize=12)
axes[1].set_ylabel('Amplitude', fontsize=12)
axes[1].set_title(f'Decision Regions (SNR = {snr_dB_demo} dB)', fontsize=14)
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=9)
axes[1].axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

# Count errors
errors_1 = np.sum(r1 < 0)  # Bit 1 sent, detected as 0
errors_0 = np.sum(r0 > 0)  # Bit 0 sent, detected as 1
total_errors = errors_1 + errors_0
ber_simulated = total_errors / (2 * num_samples)

print("\nBPSK Signal Space Analysis:")
print("═" * 70)
print(f"Amplitude: A = {A}")
print(f"Bit duration: T = {T_b*1000} ms")
print(f"Energy per bit: E_b = {E_b:.6f} J")
print(f"\nSignal coordinates:")
print(f"  Bit 1: s₁ = +{s1:.6f}")
print(f"  Bit 0: s₀ = {s0:.6f}")
print(f"  Distance: d = {distance:.6f}")
print(f"\nSimulation (SNR = {snr_dB_demo} dB, {num_samples} bits each):")
print(f"  Errors when bit 1 sent: {errors_1}")
print(f"  Errors when bit 0 sent: {errors_0}")
print(f"  Simulated BER: {ber_simulated:.6f}")
print("═" * 70)

## Part 3: BER Derivation for BPSK

### Detection Rule
Received signal: $r = s + n$ where $n \sim \mathcal{N}(0, \sigma^2)$

**Decision:**
- If $r > 0$ → Decide bit "1"
- If $r < 0$ → Decide bit "0"

### Error Probability
For BPSK, error occurs when noise crosses the decision boundary:

$$P_e = P(\text{error} | \text{bit 1}) = P(n < -A\sqrt{T_b/2})$$

With $\sigma^2 = N_0/2$ and $E_b = A^2T_b/2$:

$$P_e = Q\left(\sqrt{\frac{2E_b}{N_0}}\right)$$

This is the **fundamental BER expression for BPSK**.

In [ ]:
# BER theoretical calculation
EbN0_dB = np.arange(0, 15, 0.5)  # E_b/N_0 from 0 to 14 dB
EbN0_linear = 10**(EbN0_dB / 10)

# Theoretical BER for different modulations
BER_BPSK = Q_function(np.sqrt(2 * EbN0_linear))
BER_QPSK = Q_function(np.sqrt(2 * EbN0_linear))  # Same as BPSK
BER_ASK = Q_function(np.sqrt(EbN0_linear))  # 3 dB worse
BER_FSK_coherent = Q_function(np.sqrt(EbN0_linear))  # Same as ASK
BER_FSK_noncoherent = 0.5 * np.exp(-EbN0_linear / 2)

# Plot
plt.figure(figsize=(12, 8))
plt.semilogy(EbN0_dB, BER_BPSK, linewidth=2, label='BPSK / QPSK (Coherent)', marker='o', markersize=4)
plt.semilogy(EbN0_dB, BER_ASK, linewidth=2, label='ASK / FSK (Coherent)', marker='s', markersize=4)
plt.semilogy(EbN0_dB, BER_FSK_noncoherent, linewidth=2, label='FSK (Non-Coherent)', marker='^', markersize=4)

# Reference lines
plt.axhline(y=1e-3, color='red', linestyle='--', linewidth=1, alpha=0.5, label='BER = 10⁻³')
plt.axhline(y=1e-5, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='BER = 10⁻⁵')

plt.xlabel('E_b/N_0 (dB)', fontsize=12)
plt.ylabel('Bit Error Rate (BER)', fontsize=12)
plt.title('Theoretical BER vs E_b/N_0 for Different Modulations', fontsize=14)
plt.grid(True, which='both', alpha=0.3)
plt.legend(fontsize=10)
plt.xlim(0, 14)
plt.ylim(1e-6, 1)
plt.show()

# Find required E_b/N_0 for specific BER targets
target_bers = [1e-3, 1e-5, 1e-6]

print("\nRequired E_b/N_0 for Target BER:")
print("═" * 80)
print(f"{'Target BER':<15} {'BPSK (dB)':<15} {'ASK (dB)':<15} {'Advantage (dB)'}")
print("─" * 80)

for target_ber in target_bers:
    # BPSK: Solve Q(√(2·E_b/N_0)) = BER
    # Inverse Q-function approximation
    idx_bpsk = np.argmin(np.abs(BER_BPSK - target_ber))
    idx_ask = np.argmin(np.abs(BER_ASK - target_ber))
    
    ebn0_bpsk = EbN0_dB[idx_bpsk]
    ebn0_ask = EbN0_dB[idx_ask]
    advantage = ebn0_ask - ebn0_bpsk
    
    print(f"{target_ber:<15.0e} {ebn0_bpsk:<15.1f} {ebn0_ask:<15.1f} {advantage:<15.1f}")

print("═" * 80)
print("\nKey Observations:")
print("  • BPSK requires ~3 dB less E_b/N_0 than ASK for same BER")
print("  • QPSK has same BER as BPSK (2 bits/symbol, same bandwidth)")
print("  • Non-coherent FSK is ~1 dB worse than coherent FSK")
print("  • For BER = 10⁻⁵, BPSK needs ~9.6 dB E_b/N_0")

## Part 4: BER Simulation for BPSK

Let's verify the theoretical BER formula through Monte Carlo simulation.

In [ ]:
# BPSK BER Simulation
def simulate_bpsk_ber(EbN0_dB, num_bits=100000):
    """
    Simulate BPSK and calculate BER
    
    Parameters:
    - EbN0_dB: E_b/N_0 in dB
    - num_bits: Number of bits to simulate
    
    Returns:
    - BER: Bit error rate
    """
    EbN0_linear = 10**(EbN0_dB / 10)
    
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)
    
    # BPSK modulation: 0 → -1, 1 → +1 (unit energy symbols)
    symbols = 2 * bits - 1
    
    # Add AWGN
    # For unit energy symbols: E_s = 1
    # Noise variance: σ² = N_0/2 = 1/(2·E_b/N_0)
    noise_variance = 1 / (2 * EbN0_linear)
    noise = np.sqrt(noise_variance) * np.random.randn(num_bits)
    
    # Received signal
    received = symbols + noise
    
    # Detection: Hard decision
    detected_bits = (received > 0).astype(int)
    
    # Count errors
    errors = np.sum(bits != detected_bits)
    ber = errors / num_bits
    
    return ber

# Simulation parameters
EbN0_sim_dB = np.arange(0, 12, 1)
num_bits_sim = 100000

print(f"Simulating BPSK BER with {num_bits_sim} bits per SNR point...")
print("This may take a moment...\n")

# Run simulation
BER_sim = []
for ebn0 in EbN0_sim_dB:
    ber = simulate_bpsk_ber(ebn0, num_bits_sim)
    BER_sim.append(ber)
    print(f"E_b/N_0 = {ebn0:2d} dB: BER = {ber:.6f}")

BER_sim = np.array(BER_sim)

# Theoretical BER
EbN0_theory_linear = 10**(EbN0_sim_dB / 10)
BER_theory = Q_function(np.sqrt(2 * EbN0_theory_linear))

# Plot comparison
plt.figure(figsize=(12, 8))
plt.semilogy(EbN0_sim_dB, BER_theory, 'b-', linewidth=2, label='BPSK Theory')
plt.semilogy(EbN0_sim_dB, BER_sim, 'ro', markersize=10, label='BPSK Simulation')

plt.xlabel('E_b/N_0 (dB)', fontsize=12)
plt.ylabel('Bit Error Rate (BER)', fontsize=12)
plt.title(f'BER vs E_b/N_0 for BPSK (Simulation: {num_bits_sim} bits/point)', fontsize=14)
plt.grid(True, which='both', alpha=0.3)
plt.legend(fontsize=12, loc='upper right')
plt.xlim(0, 11)
plt.ylim(1e-5, 1)
plt.show()

print("\n" + "═" * 70)
print("Simulation Results Summary:")
print("─" * 70)
print(f"{'E_b/N_0 (dB)':<15} {'Theory':<15} {'Simulation':<15} {'Error'}")
print("─" * 70)
for ebn0, ber_th, ber_sim in zip(EbN0_sim_dB, BER_theory, BER_sim):
    error_pct = abs(ber_th - ber_sim) / ber_th * 100 if ber_th > 0 else 0
    print(f"{ebn0:<15.0f} {ber_th:<15.2e} {ber_sim:<15.2e} {error_pct:<.1f}%")
print("═" * 70)
print("\nConclusion: Simulation closely matches theory!")

## Part 5: QPSK BER Simulation

**Quadrature Phase Shift Keying (QPSK)** transmits 2 bits per symbol using 4 phases.

QPSK can be viewed as two independent BPSK channels (I and Q), so:
$$P_e(\text{QPSK}) = Q\left(\sqrt{\frac{2E_b}{N_0}}\right)$$

Same BER as BPSK, but **twice the spectral efficiency** (2 bits/symbol)!

In [ ]:
# QPSK Simulation
def simulate_qpsk_ber(EbN0_dB, num_bits=100000):
    """
    Simulate QPSK and calculate BER
    """
    EbN0_linear = 10**(EbN0_dB / 10)
    
    # Generate random bits (must be even for QPSK)
    if num_bits % 2 != 0:
        num_bits += 1
    
    bits = np.random.randint(0, 2, num_bits)
    
    # Split into I and Q channels
    I_bits = bits[0::2]  # Even indices
    Q_bits = bits[1::2]  # Odd indices
    
    # BPSK modulation on each channel
    I_symbols = 2 * I_bits - 1  # ±1
    Q_symbols = 2 * Q_bits - 1  # ±1
    
    # Normalize for unit energy per symbol (E_s = 2E_b for QPSK)
    # Each channel has energy E_b
    
    # Add AWGN to each channel
    noise_variance = 1 / (2 * EbN0_linear)
    I_noise = np.sqrt(noise_variance) * np.random.randn(len(I_symbols))
    Q_noise = np.sqrt(noise_variance) * np.random.randn(len(Q_symbols))
    
    # Received signals
    I_received = I_symbols + I_noise
    Q_received = Q_symbols + Q_noise
    
    # Detection
    I_detected = (I_received > 0).astype(int)
    Q_detected = (Q_received > 0).astype(int)
    
    # Count errors
    I_errors = np.sum(I_bits != I_detected)
    Q_errors = np.sum(Q_bits != Q_detected)
    total_errors = I_errors + Q_errors
    
    ber = total_errors / num_bits
    
    return ber

# Simulate QPSK
print("Simulating QPSK BER...\n")
BER_qpsk_sim = []
for ebn0 in EbN0_sim_dB:
    ber = simulate_qpsk_ber(ebn0, num_bits_sim)
    BER_qpsk_sim.append(ber)
    print(f"E_b/N_0 = {ebn0:2d} dB: BER = {ber:.6f}")

BER_qpsk_sim = np.array(BER_qpsk_sim)

# Plot comparison: BPSK vs QPSK
plt.figure(figsize=(12, 8))
plt.semilogy(EbN0_sim_dB, BER_theory, 'b-', linewidth=2, label='BPSK/QPSK Theory')
plt.semilogy(EbN0_sim_dB, BER_sim, 'ro', markersize=10, label='BPSK Simulation')
plt.semilogy(EbN0_sim_dB, BER_qpsk_sim, 'g^', markersize=10, label='QPSK Simulation')

plt.xlabel('E_b/N_0 (dB)', fontsize=12)
plt.ylabel('Bit Error Rate (BER)', fontsize=12)
plt.title('BER Comparison: BPSK vs QPSK', fontsize=14)
plt.grid(True, which='both', alpha=0.3)
plt.legend(fontsize=12)
plt.xlim(0, 11)
plt.ylim(1e-5, 1)
plt.show()

print("\n" + "═" * 70)
print("Key Insight: QPSK has same BER as BPSK!")
print("  • QPSK transmits 2 bits/symbol (I and Q channels)")
print("  • Each channel is independent BPSK")
print("  • Same BER, but 2× spectral efficiency")
print("  • This is why QPSK is widely used (WiFi, LTE, satellite)")
print("═" * 70)

## Part 6: Modulation Comparison Summary

Let's create a comprehensive comparison of different modulation schemes.

In [ ]:
# Comprehensive modulation comparison
EbN0_compare = np.arange(0, 14, 0.5)
EbN0_compare_linear = 10**(EbN0_compare / 10)

# BER formulas
BER_bpsk_final = Q_function(np.sqrt(2 * EbN0_compare_linear))
BER_qpsk_final = Q_function(np.sqrt(2 * EbN0_compare_linear))
BER_ask_final = Q_function(np.sqrt(EbN0_compare_linear))
BER_fsk_coh = Q_function(np.sqrt(EbN0_compare_linear))
BER_fsk_noncoh = 0.5 * np.exp(-EbN0_compare_linear / 2)

# Plot all
plt.figure(figsize=(14, 8))
plt.semilogy(EbN0_compare, BER_bpsk_final, linewidth=2.5, label='BPSK (Coherent)', color='blue')
plt.semilogy(EbN0_compare, BER_qpsk_final, '--', linewidth=2.5, label='QPSK (Coherent)', color='cyan')
plt.semilogy(EbN0_compare, BER_ask_final, linewidth=2.5, label='ASK/OOK (Coherent)', color='red')
plt.semilogy(EbN0_compare, BER_fsk_coh, linewidth=2.5, label='FSK (Coherent)', color='green')
plt.semilogy(EbN0_compare, BER_fsk_noncoh, linewidth=2.5, label='FSK (Non-Coherent)', color='orange')

# Reference BER lines
plt.axhline(y=1e-3, color='gray', linestyle=':', linewidth=1.5, alpha=0.7)
plt.axhline(y=1e-5, color='gray', linestyle=':', linewidth=1.5, alpha=0.7)
plt.text(0.5, 1.5e-3, 'BER = 10⁻³', fontsize=10, color='gray')
plt.text(0.5, 1.5e-5, 'BER = 10⁻⁵', fontsize=10, color='gray')

plt.xlabel('E_b/N_0 (dB)', fontsize=14)
plt.ylabel('Bit Error Rate (BER)', fontsize=14)
plt.title('Theoretical BER Performance Comparison', fontsize=16)
plt.grid(True, which='both', alpha=0.3)
plt.legend(fontsize=12, loc='upper right')
plt.xlim(0, 13)
plt.ylim(1e-6, 1)
plt.show()

# Summary table
print("\nModulation Performance Summary:")
print("═" * 90)
print(f"{'Modulation':<25} {'BER Formula':<35} {'E_b/N_0 for 10⁻³ BER'}")
print("─" * 90)

modulations = [
    ('BPSK (Coherent)', 'Q(√(2E_b/N_0))', BER_bpsk_final),
    ('QPSK (Coherent)', 'Q(√(2E_b/N_0))', BER_qpsk_final),
    ('ASK/OOK (Coherent)', 'Q(√(E_b/N_0))', BER_ask_final),
    ('FSK (Coherent)', 'Q(√(E_b/N_0))', BER_fsk_coh),
    ('FSK (Non-Coherent)', '0.5·exp(-E_b/(2N_0))', BER_fsk_noncoh)
]

for name, formula, ber_curve in modulations:
    idx = np.argmin(np.abs(ber_curve - 1e-3))
    ebn0_required = EbN0_compare[idx]
    print(f"{name:<25} {formula:<35} {ebn0_required:.1f} dB")

print("═" * 90)
print("\nKey Takeaways:")
print("  1. BPSK/QPSK: Best BER performance (coherent detection)")
print("  2. QPSK: Same BER as BPSK, but 2× spectral efficiency")
print("  3. ASK/FSK: 3 dB worse than BPSK (need higher SNR)")
print("  4. Non-coherent FSK: Simpler receiver, ~1 dB penalty")
print("  5. For BER = 10⁻³: BPSK needs ~6.8 dB, ASK needs ~9.8 dB")
print("\nPractical Applications:")
print("  • BPSK: Satellite, deep space, RFID")
print("  • QPSK: WiFi, LTE, satellite TV (DVB-S2)")
print("  • ASK: Optical fiber, simple radios")
print("  • FSK: Bluetooth, LoRa, legacy modems")
print("═" * 90)

## Summary and Key Takeaways

### Matched Filter
- **Optimal linear filter** for maximizing SNR in AWGN
- Impulse response: $h(t) = s(T-t)$ (time-reversed signal)
- Output SNR: $2E_s/N_0$
- Implementation: Correlation receiver

### Signal Space
- Represent signals as vectors using orthonormal basis
- **BPSK**: 1D space, two antipodal points
- **QPSK**: 2D space, four points at 90° intervals
- Distance between symbols determines BER

### BER Performance
$$\text{BPSK/QPSK: } P_e = Q\left(\sqrt{\frac{2E_b}{N_0}}\right)$$

$$\text{ASK/FSK: } P_e = Q\left(\sqrt{\frac{E_b}{N_0}}\right) \text{ (3 dB worse)}$$

### Design Guidelines
1. **Choose BPSK/QPSK** for best power efficiency
2. **QPSK preferred** over BPSK (same BER, 2× spectral efficiency)
3. **Target BER = 10⁻⁵**: Requires ~9.6 dB $E_b/N_0$ for BPSK
4. **Higher-order modulation** (16-QAM, 64-QAM) trades BER for spectral efficiency

### Practical Systems
- **WiFi**: QPSK to 1024-QAM (adaptive based on SNR)
- **LTE/5G**: QPSK to 256-QAM
- **Satellite**: QPSK, 8-PSK (power-limited)
- **Deep Space**: BPSK (maximum reliability)

### Next Topics
- Channel coding (error correction codes)
- Higher-order modulations (M-PSK, M-QAM)
- Fading channels and diversity
- Adaptive modulation and coding